In [29]:
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, Dropout # type: ignore
from tensorflow.keras.optimizers import Adam, SGD, RMSprop # type: ignore
from tensorflow.keras.callbacks import EarlyStopping # type: ignore

import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import joblib

In [30]:
# Allow the use of custom functions in the utils folder if needed
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath('..'))

In [31]:
# Function to read and parse the .tct file
def read_param_from_file(filepath):
    with open(filepath, 'r') as file:
        lines = file.readlines()
    
    # Remove any empty lines and comments
    lines = [line.strip() for line in lines if line.strip() and not line.startswith('#')]
    
    # Extract features, targets, and years
    features_line = lines[0].split(',')
    targets_line = lines[1].split(',')
    years_line = lines[2].split(',')
    
    features_from_param = [feature.strip() for feature in features_line]
    targets_from_param = [target.strip() for target in targets_line]
    years_to_pick = [int(year.strip()) for year in years_line]
    
    return features_from_param, targets_from_param, years_to_pick



In [32]:
# Initialize dataframe to read
df = pd.read_csv('../data/Bird_Migration_Custom_Data.csv')

# Remove leading and trailing whitespaces from column names
df.columns = df.columns.str.strip()

# Remove leading and trailing whitespaces from each cell
for col in df.columns:
    if pd.api.types.is_string_dtype(df[col]):
        df[col] = df[col].str.strip()

In [33]:
# Read the features, targets, and years to pick from the parameters.txt file
features_from_param, targets_from_param, years_to_pick_from_param = read_param_from_file("parameters.txt")

# Filter the DataFrame based on the specified years
df = df[df['year'].isin(years_to_pick_from_param)]

# Initialize features and targets
features = df[features_from_param]
targets = df[targets_from_param]

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.2, random_state=42)

# Standardize features
scaler_X = StandardScaler()
X_train = scaler_X.fit_transform(X_train)
X_test = scaler_X.transform(X_test)

# Standardize targets (important for deep learning)
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

In [34]:
# Initialize the model
model = Sequential()

# Add layers
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))  # Input layer
# model.add(Dropout(0.5)) # Dropout makes the MSE worse
model.add(Dense(64, activation='relu'))  # Hidden layer
# model.add(Dropout(0.5)) # Dropout makes the MSE worse
model.add(Dense(1))  # Output layer (2 outputs for longitude and latitude)

# Compile the model
optimizer = Adam(learning_rate=0.001)  # You can try different values like 0.0001 or 0.01
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=300, batch_size=8, validation_split=0.2, verbose=1)

Epoch 1/300


c:\Users\Ivan\Documents\CMA\ML Caribou Migration\Machine-Learning-Caribou-Migration\venv\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.7079 - val_loss: 0.1993
Epoch 2/300
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2318 - val_loss: 0.1694
Epoch 3/300
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1516 - val_loss: 0.1350
Epoch 4/300
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1370 - val_loss: 0.1274
Epoch 5/300
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1282 - val_loss: 0.1128
Epoch 6/300
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1184 - val_loss: 0.1090
Epoch 7/300
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1313 - val_loss: 0.1137
Epoch 8/300
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0972 - val_loss: 0.1059
Epoch 9/300
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0898 - val_loss: 0.1136
Epoch 10/300
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0950 - val_loss: 0.0967
Epoch 11/300
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0964 - val_loss: 0.1132
Epoch 12/300
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0850 - val_lo

In [35]:
# Save the model
model.save('../models/model2.h5')

In [36]:
# This cell is for having 1 output in the prediction
# Evaluate the model
loss = model.evaluate(X_test, y_test, verbose=1)
print(f'Loss on test data: {loss:.4f}')

# Make predictions
y_pred = model.predict(X_test)

# Inverse transform predictions and true values
y_test_inv = scaler_y.inverse_transform(y_test)
y_pred_inv = scaler_y.inverse_transform(y_pred)

# Calculate metrics
mae_latitude = mean_absolute_error(y_test_inv, y_pred_inv)
mse_latitude = mean_squared_error(y_test_inv, y_pred_inv)
rmse_latitude = np.sqrt(mse_latitude)
r2_latitude = r2_score(y_test_inv, y_pred_inv)

print(f'Mean Absolute Error for Latitude: {mae_latitude:.2f}')
print(f'Mean Squared Error for Latitude: {mse_latitude:.2f}')
print(f'Root Mean Squared Error for Latitude: {rmse_latitude:.2f}')
print(f'R-squared for Latitude: {r2_latitude:.2f}')

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1663 
Loss on test data: 0.1414
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Mean Absolute Error for Latitude: 5.91
Mean Squared Error for Latitude: 95.26
Root Mean Squared Error for Latitude: 9.76
R-squared for Latitude: 0.84


In [37]:
# Example input data (day, month, temperature, wind_speed)
new_data = np.array([[10, 12, 24,  2]]) 
new_data_scaled = scaler_X.transform(new_data)

# Predict using the trained model
prediction_scaled = model.predict(new_data_scaled)

# Reverse the scaling to get the original units
prediction = scaler_y.inverse_transform(prediction_scaled)

# Output the prediction
target_name = targets_from_param[0]
result = prediction[0][0]
# print("Predicted ", target_name.capitalize(), " = ", prediction[0][0])
print(f"Predicted  {target_name.capitalize()} = {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicted  Latitude = -6.607059955596924


c:\Users\Ivan\Documents\CMA\ML Caribou Migration\Machine-Learning-Caribou-Migration\venv\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
